### Segmenting and Clustering Neighbourhoods in Toronto


The project includes scraping the Wikipedia page for the postal codes of Canada and then process and clean the data for the clustering. The clustering is carried out by K Means and the clusters are plotted using the Folium Library

<h4>Import necesssary libraries</h4>

In [1]:
from bs4 import BeautifulSoup 
import requests
import pandas as pd
import regex as re
import numpy as np
from geopy.geocoders import Nominatim
import folium
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

<b>Scraping the Wikipedia page for the table of postal codes of Canada
and retreiving the URL and creating a Beautiful soup object<b>

In [5]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result=requests.get(url).text
soup=BeautifulSoup(result,'lxml')

In [6]:
#extract the data
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(',1)[1]).strip(')')).replace(' /',',').replace('(',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)


In [7]:
# Creating the dataframe
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})


In [8]:
pd.set_option('display.max_colwidth',None)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


* <b>Using the Geospatial_Coordinates dataset to get the Latitude	& Longitude</b>
* <b>Concatenate the two dataframe</b> 

In [9]:
coor=pd.read_csv('Geospatial_Coordinates.csv')
df_coor=df.join(coor.set_index('Postal Code'),on='PostalCode')
df_coor

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East",43.636258,-79.498509


* <b>Use geopy library to get the latitude and longitude values of Toronto City.</b>


In [10]:
address='Toronto, Ontario'
geolocater=Nominatim(user_agent='ny_explorer')
try:
    location=geolocater.geocode(address)
except GeocoderUnavailable:
     location=geolocater.geocode(address)
Lan=location.latitude
Lon=location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(Lan,Lon))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


* <b>Create a dataframe with Boroughs that contain the word Toronto</b>

In [11]:
def search_toronto(row):
    return bool(re.search('Toronto',row))
df_Toronto=df_coor[df['Borough'].apply(search_toronto)]
print(df_Toronto.shape)
df_Toronto.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [29]:
mydf=pd.DataFrame()
for i,j,l in zip(df_Toronto['Borough'],df_Toronto['PostalCode'],range(len(df_Toronto))):
    mydf.loc[l,'com']='{},{}'.format(i,j)
mydf

,com
0,"Downtown Toronto,M5A"
1,"Downtown Toronto,M5B"
2,"Downtown Toronto,M5C"
3,"East Toronto,M4E"
4,"Downtown Toronto,M5E"
5,"Downtown Toronto,M5G"
6,"Downtown Toronto,M6G"
7,"Downtown Toronto,M5H"
8,"West Toronto,M6H"
9,"East York/East Toronto,M4J"


#### Create a map of Toronto City with borough superimposed on top.

### Note If the map doesn't appear in the github please see the README

In [12]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[Lan, Lon], zoom_start=11)

# add markers to map
for lat, lng, borough,postal in zip(df_Toronto['Latitude'], df_Toronto['Longitude'],df_Toronto['Borough'],df_Toronto['PostalCode']):
    label = '{},{}'.format(postal, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version
<p>I hide the code so no one could use my Credentials</p>
<p>I define <b>VERSION = '20180605' & LIMIT = 100</b></p>

In [25]:
CLIENT_ID = 'R3AGFQGERPS5X4U4VMVR2PBBPZXE32T54ZRFAN5NKIHCI0FR' # my Foursquare ID
CLIENT_SECRET = 'FIT4MV1PLT1PUXXSRCE1VTYVYRLTZNBOOA33HLRAU05JDVT3'# my your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 150 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: R3AGFQGERPS5X4U4VMVR2PBBPZXE32T54ZRFAN5NKIHCI0FR
CLIENT_SECRET:FIT4MV1PLT1PUXXSRCE1VTYVYRLTZNBOOA33HLRAU05JDVT3


#### Get the top 100 venues that are  within a radius of 500 meters

* First write a function that extracts the category of the venue and then use the Foursquare API to explore the neighborhoods and segment them

In [21]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### create a new dataframe called Toronto_venues.
* using the above function to create the venues dataframe for Toronto Neighborhood that have the same postal code

In [28]:
Toronto_venues=getNearbyVenues(df_Toronto['Neighborhood'],df_Toronto['Latitude'],df_Toronto['Longitude'])

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

In [29]:
print(Toronto_venues.shape)
Toronto_venues.head()

(1591, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


#### Find out how many unique categories can be curated from all the returned venues


In [30]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 228 uniques categories.


### Analyze Each Neighborhood

* preprocessing by using one hot encoding to transform **Venue Category** to a dummies variable

In [31]:
# one hot encoding
Toronto_onehot=pd.get_dummies(Toronto_venues[['Venue Category']],prefix="",prefix_sep="")
Toronto_onehot.drop('Neighborhood',1,inplace=True)
# add neighborhood column back to the begining of the dataframe
Toronto_onehot.insert(0,'Neighborhood',Toronto_venues['Neighborhood'])
Toronto_onehot.head()

,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [32]:
Toronto_grouped=Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head()

,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.017241,0.0,0.0,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
2,"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",0.0,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
3,Central Bay Street,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.015385,0.0,0.0,0.015385,0.015385
4,Christie,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000


#### create the new dataframe and display the top 10 venues for each neighborhood.

* write a function to sort the venues in descending order.

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

* create the new dataframe and display the top 15 venues for each neighborhood.

In [39]:
num_top_venues = 15

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Restaurant,Cheese Shop,Seafood Restaurant,Beer Bar,Farmers Market,Museum,Bistro,Sporting Goods Shop,Breakfast Spot,Liquor Store,Eastern European Restaurant,Café
1,"Brockton, Parkdale Village, Exhibition Place",Café,Nightclub,Coffee Shop,Breakfast Spot,Gym,Italian Restaurant,Convenience Store,Climbing Gym,Restaurant,Burrito Place,Pet Store,Stadium,Furniture / Home Store,Bar,Bakery
2,"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",Airport Service,Airport Lounge,Airport Terminal,Boutique,Airport Food Court,Airport Gate,Harbor / Marina,Bar,Plane,Airport,Rental Car Location,Boat or Ferry,Sculpture Garden,Middle Eastern Restaurant,Modern European Restaurant
3,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Japanese Restaurant,Burger Joint,Restaurant,Salad Place,Bubble Tea Shop,Ramen Restaurant,Discount Store,Portuguese Restaurant,Poke Place,Comic Shop,Park
4,Christie,Grocery Store,Café,Park,Restaurant,Nightclub,Baby Store,Italian Restaurant,Athletics & Sports,Coffee Shop,Candy Store,Monument / Landmark,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Mexican Restaurant


### Cluster Neighborhoods

* Run k-means to cluster the neighborhood into 5 clusters

In [40]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters ,init='k-means++', n_init=10, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
set(kmeans.labels_)

{0, 1, 2, 3, 4}

* create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [42]:
# add clustering labels
# neighborhoods_venues_sorted.drop('Cluster Labels',1,inplace=True)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = df_Toronto

# merge Toronto_grouped with df_Toronto to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Pub,Park,...,Theater,Breakfast Spot,Event Space,Electronics Store,Restaurant,Bank,Chocolate Shop,Mexican Restaurant,Distribution Center,Performing Arts Venue
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Sandwich Place,Café,...,Middle Eastern Restaurant,Cosmetics Shop,Pizza Place,Japanese Restaurant,Italian Restaurant,Bookstore,Bar,Restaurant,Movie Theater,Ramen Restaurant
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Clothing Store,Cocktail Bar,...,Beer Bar,Department Store,Seafood Restaurant,Restaurant,Cosmetics Shop,Gym,Park,Farmers Market,Moroccan Restaurant,Lingerie Store
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Asian Restaurant,Pub,Trail,...,Men's Store,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Martial Arts School,Mediterranean Restaurant,Molecular Gastronomy Restaurant,Market,Malay Restaurant,Lounge
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Bakery,Restaurant,...,Seafood Restaurant,Beer Bar,Farmers Market,Museum,Bistro,Sporting Goods Shop,Breakfast Spot,Liquor Store,Eastern European Restaurant,Café
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Sandwich Place,Italian Restaurant,Café,...,Burger Joint,Restaurant,Salad Place,Bubble Tea Shop,Ramen Restaurant,Discount Store,Portuguese Restaurant,Poke Place,Comic Shop,Park
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564,0,Grocery Store,Café,Park,Restaurant,...,Baby Store,Italian Restaurant,Athletics & Sports,Coffee Shop,Candy Store,Monument / Landmark,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Mexican Restaurant
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,0,Coffee Shop,Café,Restaurant,Gym,...,Deli / Bodega,Clothing Store,Hotel,Steakhouse,Concert Hall,Bookstore,Burrito Place,Seafood Restaurant,Sushi Restaurant,American Restaurant
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,0,Pharmacy,Bakery,Park,Music Venue,...,Café,Middle Eastern Restaurant,Supermarket,Bar,Bank,Brewery,Grocery Store,New American Restaurant,Museum,Malay Restaurant
35,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106,2,Park,Convenience Store,Molecular Gastronomy Restaurant,Liquor Store,...,Malay Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark


#### visualize the resulting clusters

### Note If the map doesn't appear in the github please see the README

In [43]:
# create map of Toronto using latitude and longitude values
map_clusters = folium.Map(location=[Lan, Lon], zoom_start=11)
# set color scheme for the clusters
colors_array = cm.rainbow(np.linspace(0, 1,len(set(kmeans.labels_))))
rainbow = [colors.rgb2hex(i) for i in colors_array]
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'],Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters


#### Cluster 1

In [45]:
cluster_1=Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]
cluster_1.describe(include='all')

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
count,34,34.0,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34
unique,6,NaN,17,20,23,23,26,31,26,28,29,30,27,30,33,29,29
top,Downtown Toronto,NaN,Coffee Shop,Café,Sandwich Place,Restaurant,Café,Italian Restaurant,Restaurant,Japanese Restaurant,Fried Chicken Joint,Restaurant,Caribbean Restaurant,Grocery Store,Miscellaneous Shop,Museum,Pizza Place
freq,16,NaN,15,9,3,5,3,3,3,3,2,3,2,2,2,3,2
mean,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Cluster 2


In [46]:
cluster_2=Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]
cluster_2.describe(include='all')

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
count,1,1.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
unique,1,NaN,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
top,Central Toronto,NaN,Garden,Home Service,Monument / Landmark,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant
freq,1,NaN,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
mean,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Cluster 3

In [47]:
cluster_2=Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]
cluster_2.describe(include='all')

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
count,1,1.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
unique,1,NaN,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
top,East York/East Toronto,NaN,Park,Convenience Store,Molecular Gastronomy Restaurant,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark
freq,1,NaN,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
mean,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Cluster 4

In [48]:
cluster_4=Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]
cluster_4.describe(include='all')

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
count,2,2.0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
unique,2,NaN,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,Central Toronto,NaN,Park,Trail,Trail,Restaurant,Lingerie Store,Liquor Store,Liquor Store,Malay Restaurant,Market,Market,Martial Arts School,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Middle Eastern Restaurant
freq,1,NaN,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1
mean,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Cluster 5


In [50]:
cluster_5=Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]
cluster_5.describe(include='all')

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
count,1,1.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
unique,1,NaN,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
top,Central Toronto,NaN,Park,Bus Line,Swim School,Modern European Restaurant,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Molecular Gastronomy Restaurant
freq,1,NaN,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
mean,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
